<a href="https://colab.research.google.com/github/mangeshb1/Gen_AI_And_NLP_Experiments/blob/main/Gen_AI_And_Chatbots/create_Question_Ans_Pair_Llama2_Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = "Mangesh (MJ) B"
__copyright__ = "Copyright (C) 2024 Mangesh Bhangare"
__version__ = "1.0"

In [ ]:
!pip install -q datasets
!pip install -q pyspark

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [ ]:
## This import is needed sometimes in colab due to UTF8 format error
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
#####################################
#                                   #
#  Set Data                         #
#                                   #
#####################################

!mkdir csvs
!gdown 15cDyHNRknjr4u9xyVd0ZKDIMFO-7iFSW -O csvs/mangesh_Q_and_A_Pairs_GPT.csv

mkdir: cannot create directory ‘csvs’: File exists
Downloading...
From: https://drive.google.com/uc?id=15cDyHNRknjr4u9xyVd0ZKDIMFO-7iFSW
To: /content/csvs/mangesh_Q_and_A_Pairs_GPT.csv
100% 25.6k/25.6k [00:00<00:00, 57.2MB/s]


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a Spark session
spark = SparkSession.builder.appName("CSVtoDataFrame").getOrCreate()

# Read the CSV file into a DataFrame
csv_file_path = "/content/csvs/mangesh_Q_and_A_Pairs_GPT.csv"
raw_df = spark.read.csv(csv_file_path, header=False, inferSchema=True)

print(raw_df.show(5))

# Add an index column to each row
raw_df = raw_df.withColumn("id", F.monotonically_increasing_id())

# Create DataFrames for questions and answers
questions_df = raw_df.filter(F.col("id") % 2 == 0).select(F.col("_c0").alias("question"), "id")
answers_df = raw_df.filter(F.col("id") % 2 == 1).select(F.col("_c0").alias("answer"), "id")

# Add a new column for pairing questions and answers
questions_df = questions_df.withColumn("pair_id", (questions_df.id / 2).cast("integer"))
answers_df = answers_df.withColumn("pair_id", (answers_df.id / 2).cast("integer"))

# Join questions and answers DataFrames on the pair_id column
result_df = questions_df.join(answers_df, "pair_id").drop("pair_id", "id")

# Display the result
result_df.show(truncate=False)


+--------------------+
|                 _c0|
+--------------------+
|Who is Mangesh Bh...|
|Mangesh Bhangare ...|
|When was the info...|
|The information w...|
|Where can one fin...|
+--------------------+
only showing top 5 rows

None
+---------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|question                                                                                     |answer                                                                                                                                                                                         

In [ ]:
# Assuming result_df is your DataFrame from the previous steps
result_df = result_df.withColumn("Llama2_Format", F.concat(F.lit('<s>[INST] '), result_df.question, F.lit(' [/INST] '), result_df.answer, F.lit('</s>')))

# Display the updated DataFrame
result_df.show(truncate=False)

+---------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|question                  

In [ ]:
final_df = result_df.select("Llama2_Format")

In [ ]:
final_df.show(10,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Llama2_Format                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------

In [ ]:
# If getting Following error try <lambda>() takes 0 positional arguments but 1 was given
# !pip install multiprocess
# %reset

In [ ]:
from datasets import Dataset

final_df_pd = final_df.toPandas()
# Convert Pandas DataFrame to Hugging Face Dataset
huggingface_dataset = Dataset.from_pandas(final_df_pd)

# Display the Hugging Face Dataset
print(huggingface_dataset)

Dataset({
    features: ['Llama2_Format'],
    num_rows: 99
})


In [ ]:
huggingface_dataset.push_to_hub("theLeaf1/mangesh_99_Llama2_Format")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/theLeaf1/mangesh_99_Llama2_Format/commit/0c99ebb1605aee10f7f849f564fa01115a4aae53', commit_message='Upload dataset', commit_description='', oid='0c99ebb1605aee10f7f849f564fa01115a4aae53', pr_url=None, pr_revision=None, pr_num=None)